## Observations and Insights 

1. The Summary Statistics dataframe shows both Capomulin and Ramicane have the smallest Mean tumor size of all the drug regimen tested, with mean tumor volumes of 40.67 and 40.21 respectively. Looking further, Ramicane has a slightly lower variance 23.46 vs Capomulin at 24.94.  The small variance indicates that Ramicane data points are closer to the mean that that of Capomulin. The other drugs in their cohort showed tumor volumes that were 28% to 37% larger. 
1. The drug regimen bar charts show that both Capumulin and Ramicane have a higher number of measurements than their cohorts. A higher number of measurements could allow for improved precision of the estimates and in the strength of the study.   
1. In examining the IQR results, both Capumulin and Ramicane don't have any outliers. There is a mixed picture when looking at the lower, upper and median quartile for tumor volume. The numbers are close enough that one can't discern which drug has the best results. However, when compared to their cohorts, both had significantly lower Q1 and Q3 ranges. 
1. The boxplot paints a clear picture, visually dipicting Capumulin and Ramicane had lower tumor volumes than the cohort. The boxplot also shows that Infubinol has an outlier below its minimum IQR.
1. Using scipy pearsonr, the calculated result of 0.842, shows a correlation between mouse #j246 weight and average tumor volume. The scatter chart with line regression further confirmed that as the weight of the mouse increased, so did the tumor. This was also verified by the regression line that was calculated and overlayed on top of the scatter chart.
1. The data shows that both Capumulin and Ramicane had a significant impact in reducing tumor size over those in their cohort.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
from scipy.stats import linregress
import dataframe_image as dfi

**Raw Data**

In [ ]:
# Study data files
mouse_metadata_path = 'data/Mouse_metadata.csv'
study_results_path = 'data/Study_results.csv'

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single dataset and display
df_mouse_merge = mouse_metadata.merge(study_results, left_on = 'Mouse ID', right_on = 'Mouse ID')
df_mouse_merge.head()

**Data Verification & Dataframe Cleanup**

In [ ]:
# Checking the number of mice.
mouse_qty = len(pd.unique(df_mouse_merge['Mouse ID']))
print(mouse_qty)

# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
duplicate_mouse = df_mouse_merge.loc[df_mouse_merge.duplicated(subset=[
    'Mouse ID', 'Timepoint']), 'Mouse ID'].unique()
print(duplicate_mouse)

In [ ]:
# Optional: Get all the data for the duplicate mouse ID. 
show_duplicates = df_mouse_merge.loc[df_mouse_merge['Mouse ID'] == 'g989']
show_duplicates.head()

In [ ]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
clean_mouse = df_mouse_merge.drop_duplicates().reset_index(drop=True)
clean_df = clean_mouse[clean_mouse['Mouse ID'].isin(duplicate_mouse)==False]
clean_df.head()

In [ ]:
# Checking the number of mice in the clean DataFrame.
mouse_qty = clean_df['Mouse ID'].nunique()
mouse_qty

## Summary Statistics

In [ ]:
# Generate a summary statistics table of mean, median, variance, standard deviation, 
# and SEM of the tumor volume for each regimen
mean_stat = clean_df.groupby('Drug Regimen')['Tumor Volume (mm3)'].mean()
median_stat= clean_df.groupby('Drug Regimen')['Tumor Volume (mm3)'].median()
var_stat = clean_df.groupby('Drug Regimen')['Tumor Volume (mm3)'].var()
stdv_stat = clean_df.groupby('Drug Regimen')['Tumor Volume (mm3)'].std()
sem_stat = clean_df.groupby('Drug Regimen')['Tumor Volume (mm3)'].sem()

summary_df = pd.DataFrame({'Mean': mean_stat, 'Median': median_stat, 'Variance': 
                           var_stat, 'Std. Dev.': stdv_stat, 'SEM': sem_stat})
dfi.export(summary_df, '../Images/sum_stats.png')   # Export dataframe as image
summary_df

In [ ]:
# Create summary statistics table of mean, median, variance, standard deviation, and SEM 
# of the tumor volume for each regimen using the aggregation method, 
# produce the same summary statistics in a single line
agg_group = clean_df.groupby('Drug Regimen')
agg_summary = agg_group.agg(['mean', 'median', 'var', 'std', 'sem'])['Tumor Volume (mm3)']
agg_summary.rename(columns = {'mean':'Mean', 'median':'Median',
                              'var':'Variance', 'std':'Std. Dev.', 'sem':'SEM'}, inplace = True)
agg_summary.head()

## Bar and Pie Charts

In [ ]:
# Generate a bar plot showing the total number of measurements taken on each drug regimen using pandas.
regimen_df = clean_df.groupby(['Drug Regimen']).count().reset_index()
regimen_data = regimen_df[['Drug Regimen', 'Mouse ID']].rename(columns={'Mouse ID': 'Count'})
regimen_data = regimen_data.set_index('Drug Regimen')
regimen_data.plot(kind='bar', color='royalblue', figsize=(8,4,))

plt.grid(b=False)  # hide grid lines
plt.ylabel('Number of Measurements')

plt.gca().get_legend().remove()  # turn legend off
#plt.gca().get_bar().set_color('royalblue')
plt.title('Drug Regimen Measurements')
plt.show()

In [ ]:
# Generate a bar plot showing the total number of measurements taken on each drug regimen using pyplot.
# create drug regiment data set
regimen_list = summary_df.index.tolist()
x_axis = regimen_list

# Create regimen count
regimen_count = (clean_df.groupby(['Drug Regimen'])['Mouse ID'].count()).tolist()
fig = plt.figure(figsize =(8, 4))  # format figure Size

# Format title
plt.title('Drug Regimen Measurements')
plt.xlabel('Drug Regimen')
plt.ylabel('Number of Measurements')
plt.xticks(rotation=90)
plt.grid(b=False)   # hide grid lines

# Plot bar chart
plt.bar(x_axis, regimen_count, color='royalblue', alpha=1, width=.5, align='center')
#plt.tight_layout()

In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pandas
gender_count = clean_df.groupby('Sex')
gender_count = pd.DataFrame(gender_count['Sex'].count())

# Plot
gender_count.plot(kind='pie', y='Sex', figsize=(4, 4),\
                  title='Male vs. Female Mouse Population',startangle=140,\
                  autopct="%1.1f%%", shadow=True, fontsize=12, colors=['pink', 'royalblue'], 
                  explode=(0,0.1), legend=False)
plt.tight_layout()

In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot
gender_count = (clean_df.groupby(['Sex'])['Mouse ID'].count()).tolist()
plt.figure(figsize=(4,4))

# labels for the sections of the pie chart
labels = ['Females', 'Males']
plt.title('Male vs Female Mouse Population', fontsize = 14)

sizes = gender_count            # The values of each section of the pie chart
colors = ['#ff9999','#66b3ff']  # Colors each section of the pie chart
explode = (0.1, 0)              # Tells matplotlib not to seperate the sections

# Creates the pie chart based upon the values above and automatically finds % of each part of chart
plt.pie(sizes, explode=explode, labels=labels, colors=['pink', 'royalblue'],
        autopct="%1.1f%%", shadow=True, startangle=140)

plt.tight_layout()

## Quartiles, Outliers and Boxplots

**Prime Alternate**

In [ ]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse
df_max = clean_df.groupby('Mouse ID')['Timepoint'].max()

# Merge this group df with the original dataframe to get the tumor volume at the last timepoint
df_max = pd.merge(df_max, clean_df, on=('Mouse ID', 'Timepoint'), how='left')
df_max.head()     # Preview data table

**Find IQR using forLoop**

In [ ]:
# Put treatments into a list for for loop (and later for plot labels)
list_drugs = ['Capomulin', 'Ramicane', 'Infubinol', 'Ceftamin']

# Create empty list to fill with tumor vol data (for plotting)
empty_drugs_list = []
empty_drugs_list = 0
# Filter df_max and create subsets using list of drugs
key_drugs = df_max[df_max['Drug Regimen'].isin(list_drugs)]
#####key_drugs = key_drugs.sort_values(by = 'Drug Regimen')

# Interate over rows in key drugs dataframe and create subset using loc
for drugs in key_drugs['Drug Regimen'].unique():
    df_s = key_drugs.loc[key_drugs['Drug Regimen'] == drugs]#.drop('Drug Regimen',1)
    #print(drugs + ':\n',df_s)
#    empty_drugs_list.append(df_s)
#csv = empty_drugs_list
df_s.head()
#print(empty_drugs_list)
df_s.to_csv('./empty_drugs_list.csv', index = False)
#empty_drugs_list[0].to_csv('./empty_drugs_list.csv', index = False)
#print(empty_drugs_list)#.head()    
# Create subset for Capomulin


 #***use variable drug in place of actual drug name

 #*** store total volumes in the list. There will be 4 separate volumes, using for Loop
 #***empty_drugs_list.append() #then add the volume once its retrieved

##for drug in list_drugs:
    # Locate the rows which contain mice on each drug and get the tumor volumes
     #they want you to use the dataframe that you created in the previous cell. Create a subset for each drug

       # add subset 
      #just put the subset in the empty list you created
####*** use empty list (aka tumor volume list) to find the IQR outliers
    #then find the outliers and quantiles from the subset of the drug
 ## https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.quantile.html

In [ ]:
for drug in drug_list
capo_quar = empty_drugs_list['Tumor Volume (mm3)']
quartiles = capo_quar.quantile([.25,.5,.75])
lowerquart = quartiles[.25]
upperquart = quartiles[.75]
iqr = upperquart-lowerquart

print(f'The lower quartile of the tumor volume is: {lowerquart}')
print(f'The upper quartile of the tumor volume is: {upperquart}')
print(f'The interquartile range of the tumor volume is: {iqr}')
print(f'The the median of tumor the volume is: {quartiles[.5]} ')

# Determine outliers by using upper and lower bounds
high = upperquart + (1.5*iqr)
low = lowerquart - (1.5*iqr)    
print('\n')
print(f'Values below {low} could be outliers.')
print(f'Values above {high} could be outliers.')

**Test Cell**

In [ ]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse

# Retrieve rows of specified Drug Regimen
Capomulin_df = clean_df.loc[clean_df['Drug Regimen'] == 'Capomulin']
Ramicane_df = clean_df.loc[clean_df['Drug Regimen'] == 'Ramicane']
Infubinol_df = clean_df.loc[clean_df['Drug Regimen'] == 'Infubinol']
Ceftamin_df = clean_df.loc[clean_df['Drug Regimen'] == 'Ceftamin']

# Final tumor volume at max timepoint
capo_max = Capomulin_df.groupby('Mouse ID')['Timepoint'].max()
rami_max = Ramicane_df.groupby('Mouse ID')['Timepoint'].max()
infu_max = Infubinol_df.groupby('Mouse ID')['Timepoint'].max()
ceft_max = Ceftamin_df.groupby('Mouse ID')['Timepoint'].max()

# Merge final tumor vol with clean_df dataframe to get the tumor volume at the last timepoint
capo_merge = pd.merge(capo_max, clean_df, on= ('Mouse ID', 'Timepoint'),how='left')
rami_merge = pd.merge(rami_max, clean_df, on= ('Mouse ID', 'Timepoint'),how='left')
infu_merge = pd.merge(infu_max, clean_df, on= ('Mouse ID', 'Timepoint'),how='left')
ceft_merge = pd.merge(ceft_max, clean_df, on= ('Mouse ID', 'Timepoint'),how='left')
capo_merge.head()  # used to validate dataframe

**End of Test**

In [ ]:
# Capomulin. Calculate the IQR and quantitatively determine if there are any potential outliers. 
capo_quar = capo_merge['Tumor Volume (mm3)']
quartiles = capo_quar.quantile([.25,.5,.75])
lowerquart = quartiles[.25]
upperquart = quartiles[.75]
iqr = upperquart-lowerquart

print(f'The lower quartile of the tumor volume is: {lowerquart}')
print(f'The upper quartile of the tumor volume is: {upperquart}')
print(f'The interquartile range of the tumor volume is: {iqr}')
print(f'The the median of tumor the volume is: {quartiles[.5]} ')

# Determine outliers by using upper and lower bounds
high = upperquart + (1.5*iqr)
low = lowerquart - (1.5*iqr)    
print('\n')
print(f'Values below {low} could be outliers.')
print(f'Values above {high} could be outliers.')

In [ ]:
# Ceftamin. Calculate the IQR and quantitatively determine if there are any potential outliers. 
ceft_quar = ceft_merge['Tumor Volume (mm3)']
quartiles = ceft_quar.quantile([.25,.5,.75])
lowerquart = quartiles[.25]
upperquart = quartiles[.75]
iqr = upperquart-lowerquart

print(f'The lower quartile of the tumor volume is: {lowerquart}')
print(f'The upper quartile of the tumor volume is: {upperquart}')
print(f'The interquartile range of the tumor volume is: {iqr}')
print(f'The the median of tumor the volume is: {quartiles[.5]} ')

# Determine outliers by using upper and lower bounds
high = upperquart + (1.5*iqr)
low = lowerquart - (1.5*iqr)    
print('\n')
print(f'Values below {low} could be outliers.')
print(f'Values above {high} could be outliers.')

In [ ]:
# Ramicane. Calculate the IQR and quantitatively determine if there are any potential outliers. 
rami_quar = rami_merge['Tumor Volume (mm3)']
quartiles = rami_quar.quantile([.25,.5,.75])
lowerquart = quartiles[.25]
upperquart = quartiles[.75]
iqr = upperquart-lowerquart

print(f'The lower quartile of the tumor volume is: {lowerquart}')
print(f'The upper quartile of the tumor volume is: {upperquart}')
print(f'The interquartile range of the tumor volume is: {iqr}')
print(f'The the median of tumor the volume is: {quartiles[.5]} ')

# Determine outliers by using upper and lower bounds
high = upperquart + (1.5*iqr)
low = lowerquart - (1.5*iqr)    
print('\n')
print(f'Values below {low} could be outliers.')
print(f'Values above {high} could be outliers.')

In [ ]:
# Infubinol. Calculate the IQR and quantitatively determine if there are any potential outliers. 
infu_quar = infu_merge['Tumor Volume (mm3)']
quartiles = infu_quar.quantile([.25,.5,.75])
lowerquart = quartiles[.25]
upperquart = quartiles[.75]
iqr = upperquart-lowerquart

print(f'The lower quartile of the tumor volume is: {lowerquart}')
print(f'The upper quartile of the tumor volume is: {upperquart}')
print(f'The interquartile range of the tumor volume is: {iqr}')
print(f'The the median of tumor the volume is: {quartiles[.5]} ')

# Determine outliers by using upper and lower bounds
high = upperquart + (1.5*iqr)
low = lowerquart - (1.5*iqr)    
print('\n')
print(f'Values below {low} could be outliers.')
print(f'Values above {high} could be outliers.')

In [ ]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest
plot_regimen_data =[capo_quar, rami_quar, infu_quar, ceft_quar]
regimen_labels = ['Capomulin', 'Ramicane', 'Infubinol', 'Ceftamin']

fig1, ax = plt.subplots(figsize=(8, 4))  # Plot size
f = 'lightgray'  # Property dictionaries (colors)
c = 'blue'
b = 'lightblue'
w = 'red'
m = 'blue'
ax.set_title('Tumor Volume of Selected Drug Regimen',fontsize =18)
ax.set_ylabel('Final Tumor Volume (mm3)',fontsize = 10)
ax.set_xlabel('Drug Regimen',fontsize = 12)
ax.boxplot(plot_regimen_data, labels=regimen_labels, widths = 0.4, patch_artist=True,vert=True, 
           boxprops=dict(facecolor=f, color=c),
           capprops=dict(color=c),
           whiskerprops=dict(color=c),
           flierprops=dict(color=w, markeredgecolor=w),
           medianprops=dict(color=m),
           )

plt.ylim(10, 80)

plt.savefig("../Images/box_plot.png", bbox_inches = "tight")

plt.show()

## Line and Scatter Plots

In [ ]:
# Select mouse treated with Capomulin and display data
line_df = Capomulin_df.loc[Capomulin_df['Mouse ID'] == 'j246',:]
line_df.head()

In [ ]:
# Generate a line plot of tumor volume vs. time point for a mouse #j246
x_axis = line_df['Timepoint']
y_axis = line_df['Tumor Volume (mm3)']

fig1, ax1 = plt.subplots(figsize=(8, 4))   # Plot size
plt.plot(x_axis, y_axis,linewidth=1, markersize=10,marker="o",color="blue")
plt.title('Mouse j246 Treated with Capomulin',fontsize = 18)
plt.xlabel('Time (Days)',fontsize = 10)
plt.ylabel('Tumor Volume (mm3)',fontsize = 10)
plt.show()

## Correlation and Regression

In [ ]:
# Create scatter plot of tumor volume vs mouse weight for Capomulin treatment 
avg_capo_data = Capomulin_df.groupby(['Mouse ID']).mean()
x_val = avg_capo_data['Weight (g)']
y_val = avg_capo_data['Tumor Volume (mm3)']

fig1, ax1 = plt.subplots(figsize=(8, 4))   # Plot size
plt.scatter(x_val,y_val, s=120, color='orange')
plt.title('Mouse j246 Weight vs. Average Tumor Volume',fontsize =18)
plt.xlabel('Weight (g)',fontsize =10)
plt.ylabel('Averag Tumor Volume (mm3)',fontsize =10)
plt.show()

In [ ]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
corr, _= pearsonr(x_val,y_val)
print(f'The correlation between #j246 weight and average tumor volume is: %.3f' % corr)

# use lingress to find the values used to derive values for the linear regression model
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_val,y_val)
regression_val = x_val*slope+intercept
print('\n')
print(f'Slope: {slope}')
print(f'Intercept: {intercept}')
print(f'Rvalue (Correlation Coefficient): {rvalue}')
print(f'Scipy Pearnsonr (Correlation Coefficeint):  %.2f%%' % corr)
print(f'Pvalue: {pvalue}')
print(f'Stderr: {stderr}')

In [ ]:
# Using linear regression values, plot the linear regression model on top of the previous scatter plot. 
fig1, ax1 = plt.subplots(figsize=(8, 4))   # Plot size
plt.scatter(x_val,y_val, s=120, color='orange')
plt.title('Mouse j246 Weight vs. Average Tumor Volume',fontsize =18)
plt.xlabel('Weight (g)',fontsize =10)
plt.ylabel('Averag Tumor Volume (mm3)',fontsize =10)
plt.plot(x_val,regression_val,'b-')
plt.savefig('../Images/linear_regression.png', bbox_inches = 'tight')
plt.show()

**Challenging Myself**

In [ ]:
# Used numpy polyfit to calculate the slope-intercept for the linear regression line
fig1, ax1 = plt.subplots(figsize=(8, 4))   # Plot size
plt.scatter(x_val,y_val, s=120, color='orange')
plt.title('Mouse j246 Weight vs. Average Tumor Volume',fontsize =18)
plt.xlabel('Weight (g)',fontsize =10)
plt.ylabel('Averag Tumor Volume (mm3)',fontsize =10)

# Get slope and intercept for linear regression line (LRL)
m, b = np.polyfit(x_val,y_val, 1)

# add LRL to scatter plot
plt.plot(x_val, m*x_val+b)
plt.show()